In [7]:
import pandas as pd
import os

# Path to the main test.csv
main_test_csv = r"C:\Users\zezom\PycharmProjects\HorusEye\test.csv"

# Output folders (make sure these exist)
mi_dir = r"C:\Users\zezom\PycharmProjects\HorusEye\Data\Test\MI"
ssvep_dir = r"C:\Users\zezom\PycharmProjects\HorusEye\Data\Test\SSVEP"
os.makedirs(mi_dir, exist_ok=True)
os.makedirs(ssvep_dir, exist_ok=True)

# Load and clean columns
df = pd.read_csv(main_test_csv)
df.columns = df.columns.str.strip()

# Make sure the paradigm column exists
if 'paradigm' not in df.columns:
    raise ValueError("No 'paradigm' column in test.csv!")

# Split by paradigm
mi_df = df[df['paradigm'].str.upper() == "MI"]
ssvep_df = df[df['paradigm'].str.upper() == "SSVEP"]

# Save splits
mi_df.to_csv(os.path.join(mi_dir, "test.csv"), index=False)
ssvep_df.to_csv(os.path.join(ssvep_dir, "test.csv"), index=False)

print("✅ test.csv split into MI and SSVEP test sets!")


ValueError: No 'paradigm' column in test.csv!

In [8]:
import pandas as pd
df = pd.read_csv(r"C:\Users\zezom\PycharmProjects\HorusEye\test.csv")
print(df.columns)
print(df.head())


Index(['id', 'subject_id', 'task', 'trial_session', 'trial'], dtype='object')
     id subject_id task  trial_session  trial
0  4901        S36   MI              1      1
1  4902        S36   MI              1      2
2  4903        S36   MI              1      3
3  4904        S36   MI              1      4
4  4905        S36   MI              1      5


In [11]:
import os
import numpy as np
import pandas as pd
import pickle
from keras.models import load_model

# --- CONFIG ---
base_dir = r"C:\Users\zezom\PycharmProjects\HorusEye"
test_csv = os.path.join(base_dir, "test.csv")

mi_model_path = os.path.join(base_dir, "mi_model.keras")
mi_encoder_path = os.path.join(base_dir, "mi_label_encoder.pkl")
mi_dl_ready_dir = os.path.join(base_dir, "Data", "Test", "MI", "DL_ready")

ssvep_model_path = os.path.join(base_dir, "ssvep_model.keras")
ssvep_encoder_path = os.path.join(base_dir, "ssvep_label_encoder.pkl")
ssvep_dl_ready_dir = os.path.join(base_dir, "Data", "Test", "SSVEP", "DL_ready")

submission_csv = os.path.join(base_dir, "submission.csv")

# --- LOAD MODELS AND ENCODERS ---
mi_model = load_model(mi_model_path)
with open(mi_encoder_path, "rb") as f:
    mi_label_encoder = pickle.load(f)

ssvep_model = load_model(ssvep_model_path)
with open(ssvep_encoder_path, "rb") as f:
    ssvep_label_encoder = pickle.load(f)

# --- LOAD TEST CSV ---
df = pd.read_csv(test_csv)

# --- Helper: Predict function for a row ---
def predict_row(row, dl_ready_dir, model, label_encoder):
    subject = str(row['subject_id']).strip()
    session = str(row['trial_session']).strip()
    trial_num = int(row['trial']) - 1  # 0-based index!
    npy_path = os.path.join(dl_ready_dir, f"{subject}_{session}_EEGdata_preprocessed_DLready.npy")
    if not os.path.exists(npy_path):
        print(f"Missing EEG file for row id={row['id']}: {npy_path}")
        return 'UNKNOWN'  # You can choose a placeholder or most common class
    epochs = np.load(npy_path)
    if trial_num >= epochs.shape[0]:
        print(f"Trial {trial_num + 1} out of range for file: {npy_path}")
        return 'UNKNOWN'
    X = epochs[trial_num]  # shape: (channels, samples)
    # Transpose to (1, samples, channels) for model
    X = np.transpose(X, (1, 0))[np.newaxis, ...]
    y_pred = model.predict(X, verbose=0)
    label = label_encoder.inverse_transform([np.argmax(y_pred)])[0]
    return label

# --- Predict for all test rows ---
results = []

for idx, row in df.iterrows():
    task = str(row['task']).strip().upper()
    if task == 'MI':
        label = predict_row(row, mi_dl_ready_dir, mi_model, mi_label_encoder)
    elif task == 'SSVEP':
        label = predict_row(row, ssvep_dl_ready_dir, ssvep_model, ssvep_label_encoder)
    else:
        label = 'UNKNOWN'
        print(f"Unknown task for row id={row['id']}: {task}")
    results.append(label)

df['label'] = results

# --- Diagnostics ---
missing = df[df['label'] == 'UNKNOWN']
if not missing.empty:
    print("\nRows with missing or failed predictions:")
    print(missing[['id', 'subject_id', 'task', 'trial_session', 'trial']])

print(f"\nTotal missing predictions: {len(missing)}")

# --- Prepare submission file ---
submission = df[['id', 'label']]
submission.to_csv(submission_csv, index=False)
print(f"Submission saved: {submission_csv}")


C:\Users\zezom\miniconda3\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 21 variables whereas the saved optimizer has 40 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Trial 9 out of range for file: C:\Users\zezom\PycharmProjects\HorusEye\Data\Test\MI\DL_ready\S36_1_EEGdata_preprocessed_DLready.npy
Trial 10 out of range for file: C:\Users\zezom\PycharmProjects\HorusEye\Data\Test\MI\DL_ready\S36_1_EEGdata_preprocessed_DLready.npy
Trial 10 out of range for file: C:\Users\zezom\PycharmProjects\HorusEye\Data\Test\MI\DL_ready\S37_1_EEGdata_preprocessed_DLready.npy
Trial 9 out of range for file: C:\Users\zezom\PycharmProjects\HorusEye\Data\Test\MI\DL_ready\S38_1_EEGdata_preprocessed_DLready.npy
Trial 10 out of range for file: C:\Users\zezom\PycharmProjects\HorusEye\Data\Test\MI\DL_ready\S38_1_EEGdata_preprocessed_DLready.npy
Trial 10 out of range for file: C:\Users\zezom\PycharmProjects\HorusEye\Data\Test\MI\DL_ready\S39_1_EEGdata_preprocessed_DLready.npy
Trial 10 out of range for file: C:\Users\zezom\PycharmProjects\HorusEye\Data\Test\MI\DL_ready\S40_1_EEGdata_preprocessed_DLready.npy
Trial 4 out of range for file: C:\Users\zezom\PycharmProjects\HorusEye\

In [12]:
arr = np.load(r"C:\Users\zezom\PycharmProjects\HorusEye\Data\Test\MI\DL_ready\S36_1_EEGdata_preprocessed_DLready.npy")
print(arr.shape)  # Should be (10, channels, samples)

(8, 3, 2250)
